In [1]:
import os, time, concurrent.futures
from PIL import Image, ImageDraw, ImageFont
from google.colab import drive
import pandas as pd

drive.mount('/content/drive')

input_base = "/content/drive/MyDrive/Colab Notebooks/data_set"
output_base = "/content/output_parallel"
os.makedirs(output_base, exist_ok=True)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
def process_image(input_output_paths):
    input_path, output_path = input_output_paths
    try:
        img = Image.open(input_path).convert("RGBA")
        img_resized = img.resize((128, 128))

        watermark_text = "© Cybil "
        try:
            font = ImageFont.truetype("DejaVuSans-Bold.ttf", 12)
        except:
            font = ImageFont.load_default()

        overlay = Image.new('RGBA', img_resized.size, (255,255,255,0))
        draw = ImageDraw.Draw(overlay)

        bbox = draw.textbbox((0,0), watermark_text, font=font)
        textwidth = bbox[2] - bbox[0]
        textheight = bbox[3] - bbox[1]
        x = img_resized.width - textwidth - 6
        y = img_resized.height - textheight - 6

        draw.text((x, y), watermark_text, font=font, fill=(255,255,255,180))
        watermarked = Image.alpha_composite(img_resized, overlay).convert("RGB")
        watermarked.save(output_path)
    except Exception as e:
        print(f"Error processing {input_path}: {e}")

def get_image_paths():
    paths = []
    for root, _, files in os.walk(input_base):
        for file in files:
            if file.lower().endswith(('.png', '.jpg', '.jpeg')):
                input_path = os.path.join(root, file)
                rel_path = os.path.relpath(root, input_base)
                output_folder = os.path.join(output_base, rel_path)
                os.makedirs(output_folder, exist_ok=True)
                output_path = os.path.join(output_folder, file)
                paths.append((input_path, output_path))
    return paths


def run_parallel(num_workers, paths):
    start = time.time()
    with concurrent.futures.ProcessPoolExecutor(max_workers=num_workers) as executor:
        executor.map(process_image, paths)
    end = time.time()
    return end - start

paths = get_image_paths()
results = []

for workers in [1, 2, 4, 8]:
    t = run_parallel(workers, paths)
    results.append({"Workers": workers, "Time (s)": t})
    print(f"Completed with {workers} workers in {t:.2f} seconds")


base_time = results[0]["Time (s)"]
for r in results:
    r["Speedup"] = round(base_time / r["Time (s)"], 2)

df = pd.DataFrame(results)
print("\nSpeedup Table:\n")
print(df.to_string(index=False))

Completed with 1 workers in 34.64 seconds
Completed with 2 workers in 0.66 seconds
Completed with 4 workers in 0.58 seconds
Completed with 8 workers in 0.62 seconds

Speedup Table:

 Workers  Time (s)  Speedup
       1 34.635803     1.00
       2  0.663649    52.19
       4  0.579762    59.74
       8  0.620667    55.80
